In [1]:
import torch
import torch.nn as nn
import numpy as np
import sys
import torchvision
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.utils.data as Data
sys.path.append("..")
import d2lzh_pytorch as d2l
import torch.nn.functional as F

In [2]:
num_epochs, BATCH_SIZE = 5, 256
DOWNLOAD_MNIST = True

In [3]:
train_data = torchvision.datasets.FashionMNIST(
    root='./FashionMNIST',
    download=DOWNLOAD_MNIST,
    train=True,
    transform=transforms.ToTensor()
)

test_data = torchvision.datasets.FashionMNIST(
    root='./FashionMNIST',
    train=False,
    download=DOWNLOAD_MNIST,
    transform=transforms.ToTensor()
)

train_loader = Data.DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)
test_loader = Data.DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)


In [4]:
net = nn.Sequential(
    d2l.FlattenLayer(),
    nn.Linear(28*28, 256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 10)
)

In [5]:
for param in net.parameters():
    nn.init.normal_(param, mean=0, std=0.01)

def evaluate_accuracy():
    net.eval() #关闭dropout
    acc_test, sum_test = 0., 0
    for x, y in test_loader:
        x = Variable(x)
        out = net(x)
        out = F.softmax(out, dim=1)
        acc_test += (out.argmax(dim=1) == y).float().sum().item()
        sum_test += y.shape[0]
    net.train() #重新开启训练模式
    return acc_test, sum_test



In [6]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
# d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)


In [7]:
for epoch in range(num_epochs):
    loss_train, acc_train, sum_train = 0., 0., 0
    for step, (batch_x, batch_y) in enumerate(train_loader):
        x = Variable(batch_x)
        out = net(x)
        loss = loss_func(out, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        out = F.softmax(out, dim=1)
        # 计算准确率
        acc_train += (out.argmax(dim=1) == batch_y).float().sum().item()
        sum_train += batch_y.shape[0]
        loss_train += loss.item()

    acc_test, sum_test = evaluate_accuracy()

    print('epoch:', epoch, '|train_loss: %.4f' % (loss_train / sum_train),
          '| train_acc: %.4f' % (acc_train / sum_train), '|test_acc: %.4f' % (acc_test / sum_test))


epoch: 0 |train_loss: 0.0045 | train_acc: 0.5574 |test_acc: 0.7513
epoch: 1 |train_loss: 0.0023 | train_acc: 0.7807 |test_acc: 0.7635
epoch: 2 |train_loss: 0.0020 | train_acc: 0.8122 |test_acc: 0.7895
epoch: 3 |train_loss: 0.0018 | train_acc: 0.8345 |test_acc: 0.8557
epoch: 4 |train_loss: 0.0017 | train_acc: 0.8464 |test_acc: 0.8273
